In [2]:
import pixiedust
import numpy

Pixiedust database opened successfully
Table VERSION_TRACKER created successfully
Table METRICS_TRACKER created successfully

Share anonymous install statistics? (opt-out instructions)

PixieDust will record metadata on its environment the next time the package is installed or updated. The data is anonymized and aggregated to help plan for future releases, and records only the following values:

{
   "data_sent": currentDate,
   "runtime": "python",
   "application_version": currentPixiedustVersion,
   "space_id": nonIdentifyingUniqueId,
   "config": {
       "repository_id": "https://github.com/ibm-watson-data-lab/pixiedust",
       "target_runtimes": ["Data Science Experience"],
       "event_id": "web",
       "event_organizer": "dev-journeys"
   }
}
You can opt out by calling pixiedust.optOut() in a new cell.


Pixiedust runtime updated. Please restart kernel
Table USER_PREFERENCES created successfully
Table service_connections created successfully


In [4]:
%run ../visualisation.ipynb
%run ../read_graph.ipynb

g = read_nba_full("../../Data/Basketball/Players.csv",
                  "../../Data/Basketball/Edges.csv",
                  True, True)

all11_20 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159, 1038, 1113, 1929, 3429, 4432, 1125, 863, 1588, 2586,4198,115,2925,1459,2235,1794,415,1460,564,4168,2384,1142,3134,1849,2145,1945,3283,3669,1446,718,1031,1898,2577,2492,3502,620,3079,3672,1995,3383,4609,4090,65,2930,1504]
nba11_20 = g.subgraph(all11_20)

all15_20 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159, 1038, 1113, 1929, 3429, 4432, 1125, 863, 1588, 2586,4198,115,2925,1459,2235,1794,415,1460,564,4168,2384,1142]
nba15_20 = g.subgraph(all15_20)

all19_20 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159]
nba19_20 = g.subgraph(all19_20)

Reading full NBA graph, begin...
Reading file ../../Data/Basketball/Players.csv, begin...
Data lines read: 4800
Reading file ../../Data/Basketball/Edges.csv, begin...
Data lines read: 146369
Reading NBA vertices, begin...
Vertices read: 4800
Reading NBA edges, begin...
Weights loaded correctly
Edges read: 146369
Loading succesfull


In [5]:
def build_lpa(graph, output):
    result = graph.community_label_propagation()
    resultArray = result.membership
    i = 0
    for v in graph.vs:
        v["communities"] = [resultArray[i]]
        i += 1

    plot_full(graph, output, 1)

In [6]:
build_lpa(nba11_20, "all_stars_1120.svg")
build_lpa(nba15_20, "all_stars_1520.svg")
build_lpa(nba19_20, "all_stars_1920.svg")

Plotting graph with 83 vertices and 315 edges, begin...
Plotting sucessful
Plotting graph with 60 vertices and 133 edges, begin...
Plotting sucessful
Plotting graph with 39 vertices and 38 edges, begin...
Plotting sucessful


In [7]:
def my_basic_lpa(graph, output):
    print("Basic lpa, begin...")
    counters = []
    vertices_count = len(graph.vs)
    
    i = 0
    for v in graph.vs:
        v["communities"] = [i]
        i += 1
        counters.append(0)
    
    convergence = False
    iteration = 0
    while convergence == False and iteration < 20:
        iteration += 1
        convergence = True
        
        for idx in range(vertices_count):
            counters[idx] = 0
            
        for v in graph.vs:
            com = v["communities"][0]
            counters[com] += 1
            
        unique = sum(1 if counter > 0 else 0 for counter in counters)
        print("Iteration " + str(iteration) + ",communities " + str(unique))

        for v in graph.vs:
            current_community = v["communities"][0]   
            
            #clear counters
            for idx in range(vertices_count):
                counters[idx] = 0
            
            for e in v.all_edges():     
                if v == e.target_vertex and v == e.source_vertex:
                    print("Warn - out vertex equals current vertex")
                elif v != e.target_vertex:
                    com = e.target_vertex["communities"][0]
                    counters[com] += 1
                elif v != e.source_vertex:
                    com = e.source_vertex["communities"][0]
                    counters[com] += 1
                    
            if sum(counters) != v.degree():
                print("Warn: sum = " + str(sum(counters)) + " degree = " + str(v.degree()))
                
            max_value = max(counters)
            max_indices = []
            
            for k in range(vertices_count):
                if (counters[k] == max_value):
                    max_indices.append(k)
            
            winner_idx = max_indices[random.randrange(len(max_indices))]
        
            if current_community in max_indices:
                winner_idx = current_community
                
            if winner_idx != current_community:
                convergence = False
                
            v["communities"] = [winner_idx]
            
    plot_full(graph, output, 1.5)

In [8]:
my_basic_lpa(nba11_20, "all_stars_my_1120.svg")
my_basic_lpa(nba15_20, "all_stars_my_1520.svg")
my_basic_lpa(nba19_20, "all_stars_my_1920.svg")

Basic lpa, begin...
Iteration 1,communities 83
Iteration 2,communities 22
Iteration 3,communities 15
Iteration 4,communities 7
Iteration 5,communities 4
Iteration 6,communities 4
Plotting graph with 83 vertices and 315 edges, begin...
Plotting sucessful
Basic lpa, begin...
Iteration 1,communities 60
Iteration 2,communities 16
Iteration 3,communities 12
Iteration 4,communities 8
Iteration 5,communities 7
Plotting graph with 60 vertices and 133 edges, begin...
Plotting sucessful
Basic lpa, begin...
Iteration 1,communities 39
Iteration 2,communities 15
Plotting graph with 39 vertices and 38 edges, begin...
Plotting sucessful


In [14]:
def my_basic_lpa_overlapping(graph, max_communities, output):
    print("Basic lpa with overlapping communities, begin...")
    counters = []
    vertices_count = len(graph.vs)
    
    i = 0
    for v in graph.vs:
        v["communities"] = [(i, 1)]
        i += 1
        counters.append(0)
    
    convergence = False
    iteration = 0
    while convergence == False and iteration < 20:
        iteration += 1
        convergence = True
        
        for idx in range(vertices_count):
            counters[idx] = 0
            
        for v in graph.vs:
            for com, weight in v["communities"]:
                counters[com] += weight
            
        unique = sum(1 if counter > 0 else 0 for counter in counters)
        print("Iteration " + str(iteration) + ",communities " + str(unique))

        for v in graph.vs:
            current_communities = v["communities"]   
            
            #clear counters
            for idx in range(vertices_count):
                counters[idx] = 0
            
            for e in v.all_edges():     
                if v == e.target_vertex and v == e.source_vertex:
                    print("Warn - out vertex equals current vertex")
                elif v != e.target_vertex:
                    for com, weight in e.target_vertex["communities"]:
                        counters[com] += weight
                elif v != e.source_vertex:
                    for com, weight in e.target_vertex["communities"]:
                        counters[com] += weight
                
            max_value = max(counters)
            sorted_counters = numpy.argsort(counters)[::-1]
            new_communities = []
            
            idx = 0
            
            if max_value == 0:
                continue
                
            normalize_factor = sum(counters)
            while counters[sorted_counters[idx]] / normalize_factor >= 1 / max_communities:
                new_communities.append((sorted_counters[idx], counters[sorted_counters[idx]] / max_value))
                idx += 1
            
            if idx == 0:
                new_communities = current_communities
               
            if len(list(set(new_communities) - set(current_communities))) > 0:
                convergence = False
                
            weight_sum = sum(w for com, w in new_communities)
            for i in range(len(new_communities)):
                new_communities[i] = (new_communities[i][0], new_communities[i][1] / weight_sum)
                
            v["communities"] = new_communities
            
    
    for v in graph.vs:
        print(str(v["communities"]))
        normalized = []
        for com, _ in v["communities"]:
            normalized.append(com)
        v["communities"] = normalized
        
    plot_full(graph, output, 1.5)

In [15]:
my_basic_lpa_overlapping(nba19_20, 4, "all_stars_myo4_1920.svg")
my_basic_lpa_overlapping(nba11_20, 4, "all_stars_myo4_1120.svg")
my_basic_lpa_overlapping(nba15_20, 4, "all_stars_myo4_1520.svg")

Basic lpa with overlapping communities, begin...
Iteration 1,communities 39
Iteration 2,communities 27
Iteration 3,communities 25
Iteration 4,communities 19
Iteration 5,communities 18
Iteration 6,communities 18
Iteration 7,communities 17
Iteration 8,communities 17
Iteration 9,communities 17
Iteration 10,communities 17
Iteration 11,communities 17
Iteration 12,communities 17
Iteration 13,communities 17
Iteration 14,communities 17
Iteration 15,communities 17
Iteration 16,communities 17
Iteration 17,communities 17
Iteration 18,communities 17
Iteration 19,communities 17
Iteration 20,communities 17
[(35, 0.5), (30, 0.5)]
[(20, 0.5), (29, 0.5)]
[(22, 1.0)]
[(3, 1)]
[(4, 1)]
[(30, 1.0)]
[(32, 0.5833333333373611), (33, 0.41666666666263885)]
[(35, 1.0)]
[(24, 1.0)]
[(37, 0.5999999999933713), (32, 0.40000000000662866)]
[(30, 1.0)]
[(37, 0.5), (29, 0.5)]
[(23, 1.0)]
[(26, 1.0)]
[(37, 0.5), (26, 0.5)]
[(35, 0.5), (33, 0.5)]
[(36, 0.5), (35, 0.5)]
[(35, 1.0)]
[(18, 1)]
[(29, 1.0)]
[(20, 1.0)]
[(29, 